In [54]:
import sys
sys.path.append('..')
from PIL import Image
import numpy as np
# from src.huffman import *
from src.channel import *

In [55]:
# from itertools import batched # TO-DO: fix not working with current python version, +3.12 needed

import queue

def batched_manual(iterable, n):
    # A simple manual implementation of batched
    if n < 1:
        raise ValueError('n must be at least one')
    batch = []
    for item in iterable:
        batch.append(item)
        if len(batch) == n:
            yield tuple(batch)
            batch = []
    if batch:
        yield tuple(batch)

def build_huffman_tree(imarray, n=1):
    # Count element frequencies
    element_counts = {}
    for element in batched_manual(imarray.flatten(), n):
        if element in element_counts:
            element_counts[element] += 1
        else:
            element_counts[element] = 1

    # Build priority queue to apply Huffman algorithm
    id = {}
    idr = {}
    q = queue.PriorityQueue()
    for key, value in element_counts.items():
        print(f"{key}, ")
        id[key] = len(id)
        idr[id[key]] = key
        q.put((value, [id[key]]))

    # Build Huffman tree (transf maps id to reversed code path)
    transf = {}
    while q.qsize() > 1:
        a = q.get()
        b = q.get()
        for i in a[1]:
            if i in transf:
                transf[i] += '1'
            else:
                transf[i] = '1'
        for i in b[1]:
            if i in transf:
                transf[i] += '0'
            else:
                transf[i] = '0'
        new_val = (a[0] + b[0], a[1] + b[1])
        q.put(new_val)

    # Reverse Huffman tree paths to get actual codes (transf maps id to code)
    for key, path in transf.items():
        transf[key] = path[::-1]

    # Build decoding dictionary mapping code string to original pixel tuple
    code_to_pixel_tuple = {}
    # Assuming 'id' maps pixel tuples to element IDs, and 'transf' maps element IDs to codes
    for pixel_tuple, element_id in id.items():
         code = transf[element_id]
         code_to_pixel_tuple[code] = pixel_tuple

    return transf, id, idr, code_to_pixel_tuple

In [56]:
# Parameters
n = 14  # codeword length
k = 10   # source word length
EbfN0 = 10  # Signal to noise ratio (in times)

# Proposed parity matrix
P = np.matrix([[1, 1, 0, 0],
     [0, 1, 1, 0],
     [0, 0, 1, 1],
     [1, 0, 0, 1],
     [1, 1, 1, 0],
     [0, 1, 1, 1],
     [1, 0, 1, 1],
     [1, 1, 0, 1],
     [1, 1, 1, 1],
     [1, 0, 1, 0]], dtype=np.int64)
G = np.hstack((np.eye(k), P))
H = np.hstack((P.T, np.eye(n-k)))

In [57]:
im = Image.open('../imgs/logo.tif')
imarray = np.array(im)

x = 2

transf, id, idr, code_to_pixel_tuple = build_huffman_tree(imarray, x)

cimarray = ''
for element in batched_manual(imarray.flatten(), x):
    cimarray += transf[id[element]]

print(len(imarray))
print(len(cimarray))

# Convert the string of bits into a list of integers (0 or 1)
cimarray_list = [int(bit) for bit in cimarray]

# Convert the list into a numpy matrix (1xN row vector)
aux = np.matrix(cimarray_list)

decoded_image_bits = []

# Get the total number of bits
total_bits = aux.shape[1]

# Calculate the number of padding bits needed
padding_bits = (k - (total_bits % k)) % k

# Pad the aux matrix if necessary
if padding_bits > 0:
    padding = np.zeros((1, padding_bits), dtype=int)
    aux = np.concatenate((aux, padding), axis=1)


max_retransmissions = 100 # Set a maximum number of retransmissions to prevent infinite loops

# Process aux in chunks of size k
for i in range(0, aux.shape[1], k):
    message_chunk = aux[:, i:i+k] # This should be 1x10
    original_encoded_codeword = encode_message(message_chunk, G) # This should be 1x14

    received_codeword = None
    syndrome = np.matrix(np.ones((1, n - k), dtype=int)) # Initialize with non-zero syndrome to enter loop
    retransmission_count = 0

    # Simulate retransmissions until no error is detected or max retransmissions reached
    while np.any(syndrome) and retransmission_count < max_retransmissions:
        received_codeword = noisy_channel(original_encoded_codeword, n=n, k=k, EbfN0=EbfN0) # This should be 1x14
        syndrome = (received_codeword @ H.transpose()) % 2 # H.transpose() is 14x4, result is 1x4

        if np.any(syndrome):
            retransmission_count += 1 # Error detected, simulate retransmission

    # After successfully receiving a codeword (syndrome is zero) or max retransmissions reached
    if not np.any(syndrome):
        # This assumes the encode_message produces a systematic codeword
        decoded_message_chunk = received_codeword[:, :k] # Extract first k bits (1x10)
        decoded_image_bits.append(decoded_message_chunk)
    else:
        # If max retransmissions reached and error still detected, what to do?
        # Option 1: Append a block of zeros (treat as erasure)
        decoded_image_bits.append(np.zeros((1, k), dtype=int))
        print(f"  Max retransmissions ({max_retransmissions}) reached for chunk starting at bit {i}. Appending zero block.")
        # Option 2: Append the last received (erroneous) message part (might introduce more errors)
        # decoded_message_chunk = received_codeword[:, :k]
        # decoded_image_bits.append(decoded_message_chunk)


(True, True), 
(True, False), 
(False, False), 
(False, True), 
434
142171


In [58]:
# At this point, decoded_image_bits is a list of numpy matrices, each 1xk.
combined_decoded_bits = np.concatenate(decoded_image_bits, axis=1)

# Unpad combined_decoded_bits to get the original number of bits in cimarray
# total_bits = "length of aux before padding"
combined_decoded_bits = combined_decoded_bits[:, :total_bits]
decoded_bits_array = np.array(combined_decoded_bits).flatten()

# Now, perform inverse Huffman decoding to get back the pixel values
reconstructed_pixels = []
current_code_bits = ""
# Now, perform inverse Huffman decoding to get back the pixel values
reconstructed_pixels = []
current_code_bits = ""

# Use the code_to_pixel_tuple dictionary for decoding
# Iterate through the flattened decoded bits array
for bit in decoded_bits_array:
    current_code_bits += str(bit)
    # Check if the current accumulated bits form a valid Huffman code
    if current_code_bits in code_to_pixel_tuple:
        # Get the original pixel tuple for the found code
        pixel_tuple = code_to_pixel_tuple[current_code_bits]
        # Add the pixel values from the tuple to our list
        reconstructed_pixels.extend(pixel_tuple)
        # Reset the accumulated bits for the next code
        current_code_bits = ""

# Convert the list of reconstructed pixels to a numpy array
reconstructed_array = np.array(reconstructed_pixels, dtype=np.uint8)

# Check if the number of reconstructed pixels matches the original image size
# If not, pad or truncate the reconstructed array to match the original size
# This is necessary for reshaping but note that a size mismatch indicates errors
if reconstructed_array.size < imarray.size:
    padding_size = imarray.size - reconstructed_array.size
    reconstructed_array = np.pad(reconstructed_array, (0, padding_size), 'constant', constant_values=0)
    print(f"Padded reconstructed pixels with {padding_size} zeros to match original image size.")
elif reconstructed_array.size > imarray.size:
     reconstructed_array = reconstructed_array[:imarray.size]
     print(f"Truncated reconstructed pixels by {reconstructed_array.size - imarray.size} to match original image size.")


print(f"Reconstructed array size: {reconstructed_array.size}")
print(f"Original image size: {imarray.size}")
print(f"Reconstructed array first 20 values: {reconstructed_array[:20]}")
print(f"Reconstructed array max value: {np.max(reconstructed_array) if reconstructed_array.size > 0 else 'N/A'}")
print(f"Reconstructed array min value: {np.min(reconstructed_array) if reconstructed_array.size > 0 else 'N/A'}")
print(f"Reconstructed array mean value: {np.mean(reconstructed_array) if reconstructed_array.size > 0 else 'N/A'}")

# Count occurrences of values (optional, but can be helpful)
# unique_values, counts = np.unique(reconstructed_array, return_counts=True)
# print(f"Unique values and their counts in reconstructed_array: {list(zip(unique_values, counts))[:10]}...") # print only first 10 unique values

# Now, reshape the array back to the original image dimensions
# This step will now succeed because the array size matches imarray.size
decoded_image = reconstructed_array.reshape(imarray.shape)

# Save the decoded image as a TIFF file
from PIL import Image
# Ensure the data type is correct for Image.fromarray, typically uint8 for grayscale/color images
# Also, handle potential float values if padding introduced them, convert to int/uint8
if decoded_image.dtype != np.uint8:
    # Convert to uint8, clipping values if necessary
    decoded_image = np.clip(decoded_image, 0, 255).astype(np.uint8)


decoded_image_pil = Image.fromarray(decoded_image)
decoded_image_pil.save('../imgs/decoded.tif')
print("Image decoded and saved as 'decoded.tif'")

# You might still want to print the original reconstructed_array.size to see the extent of the mismatch
# print(f"Number of pixels reconstructed before padding/truncating: {np.array(reconstructed_pixels).size}")


Reconstructed array size: 187488
Original image size: 187488
Reconstructed array first 20 values: [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
Reconstructed array max value: 1
Reconstructed array min value: 0
Reconstructed array mean value: 0.5146142686465267
Image decoded and saved as 'decoded.tif'


In [59]:
example = np.ones((100, 100), dtype=np.uint8) * 255  # Multiply by 255 to get white pixels
example_tif = Image.fromarray(example)
example_tif.save('../imgs/decoded_white.tif')